In [1]:
import sys, os
from pathlib import Path

sys.path.append(os.path.abspath(Path().resolve().parent/'src'))

# Cinemática Inversa

In [2]:
from kinematicsrobotics.datahandler import Save, Extract
from kinematicsrobotics.dataprocessing import Preprocessing
from kinematicsrobotics.model import Model
from kinematicsrobotics.metrics import Metrics

## Base de dados

In [3]:
ext = Extract()
ext._path_project = os.path.abspath(Path().resolve().parent)

dataset = ext.dataframe( r'src\data\ready\dataset-radius-1cm.csv')
dataset

,theta_1,theta_2,theta_3,theta_4,theta_5,p_x,p_y,p_z,roll,pich,yaw
0,0,0,12,0,0,53.213314,-2.477121e-15,2.515179,3.141593e+00,-1.361357,2.752040e-16
1,0,12,12,0,0,53.606657,-1.789652e-15,13.742410,2.378531e+00,-1.570796,7.630613e-01
2,0,12,24,0,0,52.819970,-3.124530e-15,6.257590,3.141593e+00,-1.361357,2.371518e-16
3,0,12,24,12,0,53.213314,-2.915404e-15,10.000000,2.356194e+00,-1.570796,7.853982e-01
4,0,12,36,12,0,51.657132,-4.201082e-15,2.678740,3.141593e+00,-1.361357,2.500234e-16
...,...,...,...,...,...,...,...,...,...,...,...
6884,120,120,120,60,0,-9.000000,1.558846e+01,41.176915,-2.026424e-16,-0.523599,-1.047198e+00
6885,120,120,120,72,0,-7.281153,1.261133e+01,42.707475,-1.899993e-16,-0.314159,-1.047198e+00
6886,120,120,120,84,0,-5.440756,9.423666e+00,43.489851,-1.843716e-16,-0.104720,-1.047198e+00
6887,120,120,120,96,0,-3.559244,6.164791e+00,43.489851,-1.843716e-16,0.104720,-1.047198e+00


In [4]:
size_train,size_val,size_test = 0.7, 0.2, 0.1

In [5]:
data  = Preprocessing(dataset = dataset, 
                           x_labels=['p_x', 'p_y','p_z', 'roll', 'pich', 'yaw'],
                           y_labels=['theta_1', 'theta_2', 'theta_3', 'theta_4']
                        )

In [6]:
x_train, x_test, y_train, y_test = data.data_train_test

## Modelos

In [7]:
mlp = Model.mlp_regressor()


## Estimação dos hiperparâmetros

In [42]:
from sklearn.model_selection import ShuffleSplit
class ParameterOptimizer:
    def __init__(self, *, model: Model, x, y, size_train = 0.7, size_val =  0.1, size_test = 0.2, n_splits: int = 4) -> None:
        self._model = model
        self._x = x
        self._y = y
        self.size_validation(size_train = size_train, 
                             size_val = size_val,
                             size_test = size_test
        )
        self.holdout(n_splits)
    
    def size_validation(self, *, size_train, size_val, size_test):
        self._size_val, self._size_train = self.size_split_val(size_train = size_train, size_val = size_val)
        self._size_test = size_test
        
    
    # Validação cruzada hold out
    def holdout(self, n_splits):
        self._n_splits = n_splits
        
        self._cv = ShuffleSplit(n_splits=n_splits, 
                                test_size = self._size_val, 
                                random_state=42
        )

    @staticmethod
    def size_split_val(*, size_train,size_val):
        size_val = (1 - size_train/(size_train+size_val))
        size_train = 1 - size_val
        return size_val, size_train
    
class ParameterSearchMLP(ParameterOptimizer):
    def __init__(self, *, min_neurons, max_neurons, num_layers, step, **kw) -> None:
        self._min_neurons = min_neurons
        self._max_neurons = max_neurons
        self._num_layers = num_layers
        self._step = step
        super().__init__(**kw)

In [43]:
cv = ParameterOptimizer(model = mlp, 
                        x = x_train,
                        y = y_train,
                        n_splits = 4
)



In [4]:
from sklearn.model_selection import ShuffleSplit,RandomizedSearchCV,GridSearchCV
from itertools import product
from sklearn.metrics import mean_squared_error,r2_score
from pandas import DataFrame

# Calcula a porcentagem relativa dos dados de treino e validação
def size_split(size_train,size_val):
    size_val = (1 - size_train/(size_train+size_val))
    return size_val

# Espaço de busca dos neurônios
def space_hidden(min_neurons, max_neurons, layers, step):
    # Lista que armazena os neurônios
    neuron_space = []
    neuron_combinations = []
    
    # Gera a lista de possíveis números de neurônios em cada camada
    possible_neurons = list(range(min_neurons, max_neurons + 1,step))
    
    # Crie todas as combinações possíveis
    for num_layers in layers:
        neuron_combinations(list(product(possible_neurons, repeat=num_layers)))
        
    neuron_space.extend(neuron_combinations)
    return neuron_space

# Parâmetro da rede mlp
def parameter_mlp(min_neurons,max_neurons,layers,step,func_act): 
    # Dicionário que armazena os hiperparâmetros
    param_grid = {
        'hidden_layer_sizes': space_hidden(min_neurons, max_neurons,layers,step),
        'activation': func_act
    }

    return param_grid

# Random Search
def RandomizedSearch(x, y, model, param_grid, scoring,cv, n_iter):
    # Configura os parâmetros da técnica de otimização 
    random_search = RandomizedSearchCV(estimator=model, 
                                       param_distributions=param_grid, 
                                       scoring=scoring, 
                                       cv=cv, 
                                       n_iter=n_iter, 
                                       random_state=42, 
                                       return_train_score=True,
                                       verbose=0)
    
    # Treina os modelos
    random_search.fit(x, y)

    # DataFrame que armazena os resultado dos hiperparâmetros
    history = DataFrame(random_search.cv_results_)

    # Melhor hiperparâmetro
    best_estimator = random_search.best_estimator_

    return history,best_estimator

# Grid Search
def GridSearch(x, y, model, param_grid, scoring, cv):
    # Configura os parâmetros da técnica de otimização 
    gridshearch = GridSearchCV(estimator=model,
                               param_grid=param_grid,
                               cv=cv,
                               scoring=scoring,
                               verbose=0,
                               return_train_score=True)
    
    # Treina os modelos
    gridshearch.fit(x, y)
    
    # Armazena os resultado dos hiperparâmetros
    history = DataFrame(gridshearch.cv_results_)
    
    best_estimator = gridshearch.best_estimator_

    return history,best_estimator

# Validação cruzada hold out
def holdout(n_splits,test_size):
    return ShuffleSplit(n_splits=n_splits, 
                      test_size=test_size, 
                      random_state=42)



In [7]:
# Espaço de busca do grid search
min_neurons= 50
max_neurons = 300
step = 10
layers = [2,3]
func_act =  ['relu', 'tanh']

param_grid = parameter_mlp(min_neurons = min_neurons,
                           max_neurons = max_neurons,
                           layers=layers,
                           step=step,
                           func_act=func_act)

len(param_grid['hidden_layer_sizes'])

2

In [10]:
# Salvar as métricas 
sv = save(path_project[0])
path_data_save =  r'src\data\ready\history.csv'

# Dados de treino e validação
size_val = size_split(size_train,size_val)

# Espaço de busca do grid search
min_neurons= 50
max_neurons = 300
step = 10
layers = [2,3]
func_act =  ['relu', 'tanh']

param_grid = parameter_mlp(min_neurons = min_neurons,
                           max_neurons = max_neurons,
                           layers=layers,
                           step=step,
                           func_act=func_act)


# validação cruzada hold out
n_splits = 5

# Número de amostras selecionadas no espaço de busca
n_iter = 1000

# Técnica de validação
cv = holdout(n_splits,size_val)

# inicializando a rede
mlp = MLP()

# APlicação do grid search
history,best_model = RandomizedSearch(x=x_train, 
                           y=y_train, 
                           model=mlp, 
                           param_grid=param_grid,
                           scoring = 'neg_mean_squared_error', 
                           cv=cv,
                           n_iter = n_iter)


sv.dataframe(history,path_data_save)



### Infomações

In [22]:
print(f"Número de parametros de camada oculta: {len(param_grid['hidden_layer_sizes'])}")
print(f"Atributos do retorno da gridsearsh: {history.columns}")

Número de parametros de camada oculta: 204183
Atributos do retorno da gridsearsh: Index(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time',
       'param_hidden_layer_sizes', 'param_activation', 'params',
       'split0_test_score', 'split1_test_score', 'split2_test_score',
       'split3_test_score', 'split4_test_score', 'mean_test_score',
       'std_test_score', 'rank_test_score', 'split0_train_score',
       'split1_train_score', 'split2_train_score', 'split3_train_score',
       'split4_train_score', 'mean_train_score', 'std_train_score'],
      dtype='object')


In [26]:
best_model

MLPRegressor(early_stopping=True, hidden_layer_sizes=(220, 120, 210, 210),
             max_iter=1000, n_iter_no_change=5, random_state=42)

In [27]:
history[['mean_test_score','std_test_score','mean_train_score', 'std_train_score','params']]

,mean_test_score,std_test_score,mean_train_score,std_train_score,params
0,-0.005603,0.000387,-0.004849,0.000581,"{'hidden_layer_sizes': (220, 120, 210, 210), '..."


## Métricas de Avaliação

In [85]:
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

def predict_mse(y_predic,Y_test):    
    return mean_squared_error(Y_test, y_predic,multioutput='raw_values')



def curve_loss(model):
    plt.plot(model.loss_curve_)
    plt.title('Erro Quadratico')
    plt.xlabel('Épocas')
    plt.ylabel('MSE')
    plt.show()

def robo():
    # Define os parâmetros DH para cada elo
    Elos = [['theta_1',10,0,90,0],
            ['theta_2',0,18,180,0],
            ['theta_3',0,18,-180,0],

            ['theta_4',0,0,90,90],
            ['theta_5',18,0,0,0]]

    # Cria o robo por meio dos parâmetros
    return Robo('Robotest',Elos)



## Treinamento, validação e teste do modelo selecionado

In [44]:
path_data = r'src\data\ready\history.csv'
history = extract.dataframe(path_data)

history_best = history[history['rank_test_score'] == 1]

params = eval(history_best.iloc[0]['params'])

In [47]:
# Treina o modelo com hiperparâmetros selecionados
mlp = Model()

mlp.set_model(**params)

mlp.fit(x = x_train,y = y_train)



In [48]:
mlp.model

MLPRegressor(early_stopping=True, hidden_layer_sizes=(120, 220, 290),
             max_iter=1000, n_iter_no_change=5, random_state=42)

### Espaço das juntas